# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [9]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

preTrainedModel = VGG19()
preTrainedModel.trainable = False
preTrainedModel.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Check basic VGG19 Classification

In [10]:
testPicture = load_img('../noDeerDay.JPG', target_size=(224,224))
# Convert Image pixels to a numpy array to be operated on
pictureArray = img_to_array(testPicture)
# reshape Data for model
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
# prepare the image for the VGG model
pictureArray = preprocess_input(pictureArray)
# predict the probability across all output classes
yhat = preTrainedModel.predict(pictureArray)
# convert the probabilities to class labels
labels = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = labels[0][0]
# print the classification
print(labels)



1/1 [==============================] - 0s 200ms/step
[[('n04371774', 'swing', 0.16665962), ('n03967562', 'plow', 0.06998367), ('n04208210', 'shovel', 0.067548245), ('n04604644', 'worm_fence', 0.06564355), ('n03141823', 'crutch', 0.04753662)]]


2023-10-02 20:28:18.876890: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


## Create complete Model with Custom output Layer

In [11]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

completeModel = models.Sequential()
completeModel.add(preTrainedModel)
completeModel.add(Dense(100, activation = "relu"))
completeModel.add(Dense(2, activation = "softmax"))
completeModel.summary()
completeModel.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense_2 (Dense)             (None, 100)               100100    
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
Total params: 143,767,542
Trainable params: 100,302
Non-trainable params: 143,667,240
_________________________________________________________________


## Orgainze test and training data

In [12]:
trainImages = keras.utils.image_dataset_from_directory( #Takes given directory and automatically organizes it based on subdirectories
    directory='trainingData',
    labels="inferred",
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))
testImages = keras.utils.image_dataset_from_directory(
    directory='testData',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))

Found 3509 files belonging to 2 classes.
Found 305 files belonging to 2 classes.


## Prediction before training

In [13]:
testPicture = load_img('../deerdownload.jpg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

1/1 [==============================] - 0s 204ms/step
[[0.49757993 0.50242007]]


2023-10-02 20:28:19.403683: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


## Train Last two layers

In [14]:
completeModel.fit(trainImages, epochs=20, batch_size=100)

Epoch 1/20


2023-10-02 20:28:19.857083: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


110/110 [==============================] - 36s 319ms/step - loss: 0.5106 - accuracy: 0.8669
Epoch 2/20
110/110 [==============================] - 35s 315ms/step - loss: 0.2479 - accuracy: 0.9185
Epoch 3/20
110/110 [==============================] - 35s 315ms/step - loss: 0.1921 - accuracy: 0.9288
Epoch 4/20
110/110 [==============================] - 35s 316ms/step - loss: 0.1684 - accuracy: 0.9342
Epoch 5/20
110/110 [==============================] - 35s 317ms/step - loss: 0.1528 - accuracy: 0.9419
Epoch 6/20
110/110 [==============================] - 35s 315ms/step - loss: 0.1418 - accuracy: 0.9461
Epoch 7/20
110/110 [==============================] - 35s 315ms/step - loss: 0.1335 - accuracy: 0.9476
Epoch 8/20
110/110 [==============================] - 35s 316ms/step - loss: 0.1260 - accuracy: 0.9510
Epoch 9/20
110/110 [==============================] - 35s 315ms/step - loss: 0.1202 - accuracy: 0.9535
Epoch 10/20
110/110 [==============================] - 35s 315ms/step - loss: 0.1152

## Test Some data

In [15]:
score = completeModel.evaluate(testImages)
print ("loss = ", score[0])
print ("accuracy = ", score[1])

2023-10-02 20:39:58.814051: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 4s 332ms/step - loss: 0.4624 - accuracy: 0.8820
loss =  0.4623938500881195
accuracy =  0.8819671869277954


## See what happens with a deer image

In [16]:
testPicture = load_img('../deerdownload.jpg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)


# 0 is deer 1 is no deer, i need to figure out how to change this

1/1 [==============================] - 0s 25ms/step
[[9.9954605e-01 4.5396623e-04]]


## Save the model for export

In [17]:
completeModel.save("alphaModel.keras")